# GPA Prediction Starter Notebook

Welcome to the GPA Prediction Starter Notebook for our project! 🚀

In this notebook, you'll find a ready-to-use Python script that provides a solid foundation for building a GPA predictor based on the data from `year1_gpa.csv`.

## Getting Started

To get started, follow these steps:

1. **Clone the Repository**: Begin by cloning this repository to your local machine.

2. **Organize Your Data**: Ensure that your GPA data is organized in the `Data` directory, particularly the `year1_gpa.csv` file.

3. **Open the Notebook**: Open this notebook in a Jupyter environment.

4. **Follow the Code**: The notebook contains commented code that guides you through the process of setting up the data, building and training the model, and evaluating its performance.

5. **Experiment and Contribute**: Feel free to experiment with different models,engineering features, hyperparameters, or preprocessing techniques. If you come up with improvements, consider contributing them back to the project!

## Important Notes

- Ensure that you have the necessary Python libraries, such as Pandas, NumPy, and scikit-learn, installed in your environment.
- If you encounter any issues or have questions, don't hesitate to reach out. We're here to help!

Happy coding, and let's build an amazing GPA predictor together! 


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import warnings
import joblib
# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Ignore warnings
warnings.filterwarnings('ignore')
print("Importation complete")

Importation complete


In [10]:
# Load the GPA data from year1_gpa.csv
data_path = "../Data/year1_gpa.csv"  # Adjust the path as needed
gpa_data = pd.read_csv(data_path,encoding='latin1')
gpa_data.columns

FileNotFoundError: [Errno 2] No such file or directory: '../Data/year1_gpa.csv'

## Data Preprocessing
In the preprocessing stage, we carefully handle the GPA dataset by addressing missing values, performing feature engineering, and ensuring uniform data formatting to prepare it for accurate model training and prediction.

In [9]:
# changes column names to lowercase and replaces spaces with underscores
gpa_data.columns = gpa_data.columns.str.lower().str.replace(' ', '_')

NameError: name 'gpa_data' is not defined

In [ ]:


# Dictionary to map long column names to short names
new_column_names = {
    'What was your age in Year One': 'age_in_year_one',
    'Do you have a disability?': 'has_disability',
    'Did you attend extra tutorials? ': 'attended_tutorials',
    'How would you rate your participation in extracurricular activities (tech, music, partying, fellowship, etc.) in Year One?': 'extracurricular_participation',
    'How would you rate your class attendance in Year One': 'class_attendance_rating',
    'How well did you participate in class activities (Assignments, Asking and Answering Questions, Writing Notes....)': 'class_participation_rating',
    'Did you use extra materials for study in Year One? (Youtube, Other books, others)': 'used_extra_study_materials',
    'Morning': 'morning_study',
    'Afternoon': 'afternoon_study',
    'Evening': 'evening_study',
    'Late Night': 'late_night_study',
    'How many days per week did you do reading on average in Year One?': 'days_per_week_reading',
    'On average, How many hours per day was used for personal study in Year One': 'hours_per_day_personal_study',
    'Did you teach your peers in Year One': 'taught_peers',
    'How many courses did you offer in Year One?': 'courses_offered',
    'Did you fall sick in Year One? if yes, How many times do you remember (0 if none)': 'times_fell_sick',
    'What was your study mode in Year 1': 'study_mode',
    'Did you study the course your originally applied for?': 'studied_original_course',
    'Rate your financial status in Year One': 'financial_status_rating',
    'Rate the teaching style / method of the lectures received in Year One': 'teaching_style_rating',
    'What type of higher institution did you attend in Year One\n': 'institution_type',
    'What was your CGPA in Year One?': 'cgpa_year_one',
    'What grading system does your school use ( if others, type numbers only)': 'grading_system'
}

# Rename columns using the dictionary
gpa_data.rename(columns=new_column_names, inplace=True)

# Print the DataFrame with updated column names
gpa_data


In [ ]:
# List of columns to drop
columns_to_drop = ['start_time', 'completion_time', 'email', 'name', 'last_modified_time']

# Drop the specified columns
gpa_data.drop(columns=columns_to_drop, inplace=True)

# Print the DataFrame after dropping columns
gpa_data

In [ ]:
# Separate columns into numeric and categorical
numeric_columns = gpa_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = gpa_data.select_dtypes(include=[np.object]).columns.tolist()

# Print the lists
print("Numeric Columns:")
print(numeric_columns)

print("\nCategorical Columns:")
print(categorical_columns)


In [ ]:


# Ordinal encoding map
ordinal_encoding_map = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 6}

# Features to encode
features_to_encode = ['english', 'maths', 'subject_3', 'subject_4', 'subject_5']

# Apply ordinal encoding for the specified features
gpa_data[features_to_encode] = gpa_data[features_to_encode].apply(lambda col: col.map(ordinal_encoding_map))

# Perform label encoding for other categorical columns
categorical_columns = gpa_data.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()

for col in categorical_columns:
    gpa_data[col] = label_encoder.fit_transform(gpa_data[col])

# Create GPA_normalised and drop unnecessary columns
gpa_data['GPA_normalised'] = gpa_data['cgpa_year_one'] / gpa_data['grading_system']
gpa_data.drop(['grading_system', 'cgpa_year_one'], axis=1, inplace=True)


# Print the DataFrame after engineering
gpa_data


### Mutual Information

In [ ]:
from sklearn.metrics import mutual_info_score

def mutual_information_score(series):
    return mutual_info_score(series, gpa_data.GPA_normalised)

In [ ]:
gpa_data[categorical_columns].apply(mutual_information_score).sort_values(ascending=False)

### Correlation

In [ ]:
gpa_data[numeric_columns].corrwith(gpa_data.GPA_normalised)

## Machine Learning Modeling

In this section, we will walk through the steps involved in building and evaluating a machine learning model for our GPA prediction task.


### Model Training

In [ ]:
X = gpa_data.drop(['id', 'GPA_normal'], axis=1)  # Features excluding 'id' and 'GPA_normal'
y = gpa_data['GPA_normal']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

### Model Evaluation

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error (RMSE):', rmse)

### Save the model

In [ ]:
# Save the model to a file
model_filename = 'linear_regression_model.joblib'
joblib.dump(model, model_filename)

print('Model saved to', model_filename)

### Tips to Improve Model Performance

1. **Data Quality:**
   - Ensure clean, high-quality data without missing values or outliers.

2. **Feature Engineering:**
   - Create relevant and new features that capture essential patterns in the data.

3. **Model Selection:**
   - Choose appropriate models and tune hyperparameters for better performance.

4. **Ensemble Learning:**
   - Combine multiple models to improve accuracy and robustness.

5. **Regularization:**
   - Implement regularization to prevent overfitting.

7. **Domain Understanding:**
   - Understand the problem domain to make informed model decisions.

8. **Feedback Loop:**
   - Continuously iterate and improve the model based on feedback and new data.

---

## HAPPY HACKING!!


